In [ ]:
# Set OpenAI API key
os.environ["OPENAI_API_KEY"] = xxx

# Define path to the cleaned text file
path = 'doc.txt'

# Load document and create vectorstore index once for faster querying
loader = TextLoader(path)
docs = loader.load()
index = VectorstoreIndexCreator().from_loaders([loader])

# Implement conversation retrieval chain, a langchain pipeline for storing a conversation's history. 
# Specify the default 3.5 turbo model. Use GPT-4 for better quality results, but responses
# Find the top 2 important bits of context.
qa = ConversationalRetrievalChain.from_llm(llm=ChatOpenAI(model="gpt-3.5-turbo"), retriever=index.vectorstore.as_retriever(search_kwargs={"k": 2}))

# List to store the history of chats in the format [(question, answer), ...]
chat_history = []

# Convert the list chat_history to a list of tuples for easy manipulation. Gradio chat history expects list of tuples, rather than just list.
chat_history_tuples = []
for message in chat_history:
    chat_history_tuples.append((message[0], message[1]))

In [ ]:
# Define the chat interface using Gradio
with gr.Blocks() as demo:
    # Display a title for the chat interface.
    gr.Markdown('# Chat: Ask questions about xxx')
    
    # Create a chatbot component to display the chat history.
    chatbot = gr.Chatbot()
    
    # Create a textbox for users to input their questions.
    msg = gr.Textbox("Enter a question here!")
    
    # Create a button for users to clear the chat history.
    clear = gr.Button("Click here to clear chat history")
    
    # Reset the chat history when starting a new session.
    chat_history = []

    # Primary function for chatbot interface
    def user(query, chat_history):
        """
        Process the user's query and return an answer.
        
        Parameters:
        - query (str): The question input by the user.
        - chat_history (list): List to maintain a running record of the chat.
        
        Returns:
        - tuple: Empty string (placeholder for the UI) and updated chat history.
        """
        
        # Convert the chat_history to a list of tuples for processing.
        chat_history_tuples = []
        for message in chat_history:
            chat_history_tuples.append((message[0], message[1]))

        # Call the `qa` function with the user's query and chat history to get an answer.
        result = qa({"question": query, "chat_history": chat_history_tuples})

        # Append the user's question and the received answer to the chat history.
        chat_history.append((query, result["answer"]))

        return gr.update(value=""), chat_history

    # Trigger the `user` function when a question is submitted in the textbox.
    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False)
    
    # Clicking the clear button clears chat history.
    clear.click(lambda: None, None, chatbot, queue=False)

# Launch the Gradio chat interface with debugging enabled and shareability.
demo.queue().launch(debug=True)